In [1]:
# installing libraries

import MySQLdb
import pandas as pd

In [2]:
# importing Data


df = pd.read_csv(r'C:\Users\immor\MTAyoutubeapi.csv')

df.head()

,video_id,video_title,upload_date,view_count,like_count
0,gvjjSjbeb1c,How to Night Crab in Oregon - Netted 30+ Dungies,2021-11-12,923,23
1,UxlPGYiOXho,Netarts Bay Crabbing - Almost Limit Out,2021-11-11,1390,18
2,NdNqAwelLJI,Crabbing along the Oregon Coast - 100+ crabs,2021-11-11,1800,28
3,cIjmEpdGZrw,Crabbing for Lunch at Nehalem Bay - $17.50 lb,2021-08-01,767,20
4,sx425fsBU4Y,Harvesting Mussels on the Oregon Coast - Netar...,2021-07-31,387,8


In [6]:
# Option 1 to connect to database

def connect_to_db(host_name, dbname, username, password):
    try:
        conn = MySQLdb.connect(host=host_name, db=dbname, user=username, passwd=password)
    except MySQLdb.OperationalError as e:
        raise e
    else:
        print('Connected!')
    return conn

In [12]:
host_name = 'this is def the host'
dbname = 'Dogs are cool'
port = 'portland'
username = 'king'
password = 'please'

conn = connect_to_db(host_name, dbname, username, password)

In [14]:
# # Option 2 to connect to database


host_name = 'database-yt2.culiarwmtbjz.us-west-1.rds.amazonaws.com'
dbname = 'youtubedb'
port = '3306'
username = 'carsontt'
password = 'Carson4003!'


db = MySQLdb.connect(host = host_name,    # your host, usually localhost
                     user = username,     # your username
                     passwd = password,   # your password
                     db = dbname)         # name of the data base

In [98]:
# creating table

def create_table(curr):
    create_table_command = ("""CREATE TABLE IF NOT EXISTS videos (
                    video_id VARCHAR(255) PRIMARY KEY,
                    video_title TEXT NOT NULL,
                    upload_date DATE NOT NULL,
                    view_count INTEGER NOT NULL,
                    like_count INTEGER NOT NULL,
            )""")
    curr.execute(create_table_command)

In [59]:
# check if video exists


def check_if_video_exists(curr, video_id):
    query = ("""SELECT video_id 
                FROM videos 
                WHERE video_id = %s""")
    curr.execute(query, (video_id,))
    
    return curr.fetchone() is not None


In [80]:
# update row if video exists


def update_row(curr, video_id, video_title, view_count, like_count):
    query = ("""UPDATE videos
            SET video_title = %s,
                view_count = %s,
                like_count = %s,
            WHERE video_id = %s;""")
    vars_to_update = (video_title, view_count, like_count, video_id)
    curr.execute(query, vars_to_update)

In [68]:
# check if video exists


def update_db(curr,df):
    temp_df = pd.DataFrame(columns=['video_id', 'video_title', 'upload_date', 'view_count',
                                    'like_count'])

    for i, row in df.iterrows():
        if check_if_video_exists(curr, row['video_id']): # if video already exists then we will update
            update_row(curr, row['video_id'], row['upload_date'], row['video_title'], row['view_count'], row['like_count'])
        else: # The video doesn't exists so we will append to the db table
            temp_df = temp_df.append(row)
    return temp_df

In [87]:
# write insert command


def insert_into_table(curr, video_id, video_title, upload_date, view_count, like_count):
    insert_into_videos = ("""INSERT INTO videos (video_id, video_title, upload_date, view_count, like_count)
                        VALUES(%s,%s,%s,%s,%s);""")
    row_to_insert = (video_id, video_title, upload_date, view_count, like_count)
    curr.execute(insert_into_videos, row_to_insert)

In [82]:
def append_from_df_to_db(curr,df):
    for i, row in df.iterrows():
        insert_into_table(curr, row['video_id'], row['video_title'], row['upload_date'], row['view_count'], row['like_count'])

In [93]:
curr = conn.cursor()

In [94]:
create_table(curr)

In [95]:
new_vid_df = update_db(curr, df)

new_vid_df

,video_id,video_title,upload_date,view_count,like_count
0,gvjjSjbeb1c,How to Night Crab in Oregon - Netted 30+ Dungies,2021-11-12,923,23
1,UxlPGYiOXho,Netarts Bay Crabbing - Almost Limit Out,2021-11-11,1390,18
2,NdNqAwelLJI,Crabbing along the Oregon Coast - 100+ crabs,2021-11-11,1800,28
3,cIjmEpdGZrw,Crabbing for Lunch at Nehalem Bay - $17.50 lb,2021-08-01,767,20
4,sx425fsBU4Y,Harvesting Mussels on the Oregon Coast - Netar...,2021-07-31,387,8
5,VycTbbXMFxg,Oregon Coast Gaper Clamming at Netarts Bay - D...,2021-07-31,227,5
6,83jAELDjdWA,Haleakala Downhill Biking in Maui - 6500&#39;,2021-06-21,144,6
7,2XA8dRe7DAQ,Harvesting Mussels and Gaper Clams at Netarts Bay,2021-06-03,474,11
8,xj54enWsbwo,87 Gapers at Netarts Bay in an Hour,2021-06-02,715,14
9,yduuo5NnJL8,Snorkeling Kapalua Bay in Maui - Spotted Half ...,2021-04-10,250,12


In [96]:
append_from_df_to_db(curr, new_vid_df)

In [97]:
conn.commit

<function Connection.commit>